In [1]:
import whisper
import os
from pydub.utils import which

In [2]:
# Set the path to ffmpeg
os.environ["PATH"] += os.pathsep + r"C:\Users\Karim\.conda\envs\speech\Library\bin"
print(which("ffmpeg"))  # This should print the path to the ffmpeg executable

C:\Users\Karim\.conda\envs\speech\Library\bin\ffmpeg.exe


In [3]:
# Add MFA path to the PATH environment variable
mfa_path = r'C:\Users\Karim\.conda\envs\speech\Scripts'
os.environ['PATH'] += os.pathsep + mfa_path
print(which("mfa"))  # This should print the path to the ffmpeg executable

C:\Users\Karim\.conda\envs\speech\Scripts\mfa.exe


In [4]:
transcribe_model = whisper.load_model('base')
audio = whisper.load_audio('beforewav.wav')

In [5]:
audio = whisper.pad_or_trim(audio)

mel = whisper.log_mel_spectrogram(audio).to(transcribe_model.device)

_,probs = transcribe_model.detect_language(mel)
print(f"Detected Language: {max(probs,key=probs.get)}")


options = whisper.DecodingOptions()
result = whisper.decode(transcribe_model,mel,options)

print(result.text)

Detected Language: en
What the fuck did DJs actually do?


In [6]:
with open("corpus/beforewav.txt", 'w') as file:
    file.write(result.text)

In [7]:
from IPython.display import Audio
Audio('beforewav.wav')

In [8]:
from pydub import AudioSegment

# Load and convert the audio file to 16-bit PCM WAV with 16 kHz sample rate
audio = AudioSegment.from_file("beforewav.wav")
audio = audio.set_frame_rate(16000).set_sample_width(2).set_channels(1)
audio.export("corpus/converted_audio.wav", format="wav")

<_io.BufferedRandom name='corpus/converted_audio.wav'>

In [16]:
input_folder = 'corpus'  
output_path = 'output'  
dictionary_path = 'english_us_mfa.dict' 
model_path = 'english_mfa.zip'  


if not os.path.exists(output_path):
    os.makedirs(output_path)

# Run MFA alignment
!mfa align --clean {input_folder} {dictionary_path} {model_path} {output_path}

Traceback (most recent call last):
  File "C:\Users\Karim\.conda\envs\speech\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\Karim\.conda\envs\speech\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\Karim\.conda\envs\speech\Scripts\mfa.exe\__main__.py", line 4, in <module>
  File "C:\Users\Karim\.conda\envs\speech\lib\site-packages\montreal_forced_aligner\__init__.py", line 4, in <module>
    import montreal_forced_aligner.acoustic_modeling as acoustic_modeling
  File "C:\Users\Karim\.conda\envs\speech\lib\site-packages\montreal_forced_aligner\acoustic_modeling\__init__.py", line 7, in <module>
    from montreal_forced_aligner.acoustic_modeling.base import AcousticModelTrainingMixin  # noqa
  File "C:\Users\Karim\.conda\envs\speech\lib\site-packages\montreal_forced_aligner\acoustic_modeling\base.py", line 19, in <module>
    from montreal_forced_aligner.abc import MfaWorker, ModelExporterMi

In [ ]:
import parselmouth
from parselmouth.praat import call

# Define TextGrid path
textgrid_path = 'output/beforewav.TextGrid' 

# Load TextGrid file
tg = parselmouth.read(textgrid_path)

# Get the number of tiers
num_tiers = call(tg, "Get number of tiers")

# Print each tier name
for i in range(1, num_tiers + 1):
    tier_name = call(tg, "Get tier name", i)
    print(f"Tier {i}: {tier_name}")

In [ ]:
tier_order = 2
tier_name = call(tg, "Get tier name", tier_order)
num_intervals = call(tg, "Get number of intervals", tier_order)

intervals = []
for i in range(1, num_intervals + 1):
    start_time = call(tg, "Get start time of interval", tier_order, i)
    end_time = call(tg, "Get end time of interval", tier_order, i)
    text = call(tg, "Get label of interval", tier_order, i)
    intervals.append((start_time, end_time, text))

print(f"Extracted {len(intervals)} intervals from tier: {tier_name}")

In [ ]:
intervals

In [ ]:
def get_syllables(phoneme_intervals):
    vowels = ['a','e','i','o','u','y'
        'ɪ', 'ɪ', 'e', 'æ', 'ʌ', 'ɒ', 'ʊ', 'ə', 'ɐ', 'iː', 'uː', 'ɔː', 'ɜː', 'ɑː',
        'eɪ', 'aɪ', 'ɔɪ', 'aʊ', 'əʊ', 'ɪə', 'eə', 'ʊə', 'ɨ', 'ʉ', 'ø', 'y', 'ɶ',
        'æː', 'ɒː', 'ɵ', 'ɜ', 'ɘ','ə', 'ɞ', 'ʌː', 'ɯ', 'ʉː', 'æi', 'ɑi', 'ui'
    ]
    syllable_boundaries = []
    current_syllable = []
        
    start_of_word = False
    for i in range(len(phoneme_intervals)): 
        
        #if phoneme_intervals[i][2].strip() == '':
        #    start_of_word = True
        
        if phoneme_intervals[i][2].strip() != '':
            current_syllable.append((phoneme_intervals[i][0], phoneme_intervals[i][1], phoneme_intervals[i][2]))
            if any(vowel in phoneme_intervals[i][2] for vowel in vowels):
                # A vowel phoneme indicates a syllable boundary
                
                #if start_of_word == True:
                #    current_syllable.append((phoneme_intervals[i+1][0], phoneme_intervals[i+1][1], phoneme_intervals[i+1][2]))
                #    syllable_boundaries.append(current_syllable)
                #    current_syllable = []
                #else:       
                syllable_boundaries.append(current_syllable)
                current_syllable = []
        
            

    # If there are leftover phonemes, add them as the last syllable
    if current_syllable:
        if len(current_syllable) > 1:
            syllable_boundaries.append(current_syllable)
        else:
            syllable_boundaries[-1].append(current_syllable[0])

    # Combine syllable intervals
    syllable_intervals = [(syllable[0][0], syllable[-1][1], ' '.join(p[2] for p in syllable)) for syllable in syllable_boundaries]
    
    
    for start, end, syllable in syllable_intervals:
        print(f"Syllable: {syllable}, Start: {start}, End: {end}")
    
    return syllable_intervals




In [ ]:
syllable_intervals = get_syllables(intervals)

In [ ]:
# Convert syllable boundaries to milliseconds
syllable_indices_ms = [(int(start * 1000), int(end * 1000)) for start, end, text in syllable_intervals if text.strip() != '']

In [ ]:
from pydub import AudioSegment
from IPython.display import Audio, display
import os

# Define paths
input_audio_path = 'corpus/beforewav.wav'  # Replace with the actual path to your audio file
output_folder = 'output/syllables'  # Folder to save the syllable audio files

# Load the original audio using pydub
audio = AudioSegment.from_wav(input_audio_path)

# Function to split and save syllables
def split_and_save_syllables(audio, indices_ms, output_folder='output/syllables'):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    syllables = []
    for i, (start_ms, end_ms) in enumerate(syllable_indices_ms):
        syllable = audio[start_ms:end_ms]
        print(len(syllable))
        syllables.append(syllable)
        syllable.export(f'{output_folder}/syllable_{i + 1}.wav', format='wav')
    return syllables

# Split and save syllables
syllables = split_and_save_syllables(audio, syllable_indices_ms)
print("Syllables have been split and saved.")



In [ ]:
from pydub import AudioSegment
import os
from natsort import natsorted


syllable_folder = 'output/syllables' 
silence_duration_ms = 300  

syllable_files = natsorted([os.path.join(syllable_folder, f) for f in os.listdir(syllable_folder) if f.endswith('.wav')])



# Create silence audio segment
silence = AudioSegment.silent(duration=silence_duration_ms)

# Concatenate syllables with silence in between
concatenated_audio = AudioSegment.empty()
for syllable_file in syllable_files:
    syllable = AudioSegment.from_wav(syllable_file)
    concatenated_audio += syllable + silence

# Export the concatenated audio
output_file = 'output/syllables/concatenated/concatenated_syllables.wav'
concatenated_audio.export(output_file, format='wav')
print(f"Concatenated audio saved as {output_file}")

